# Data Mining Project
---

Εισαγωγή Βιβλιοθηκών & Φόρτωση

In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Φόρτωση δεδομένων
# Ορίζουμε το '?' ως NaN
df = pd.read_csv('household_power_consumption.txt', sep=';', na_values=['?'], low_memory=False)
print(f"Αρχικά missing values:\n{df.isna().sum()}")

Αρχικά missing values:
Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


Καθαρισμός & Interpolation (Χωρίς Warnings)

In [18]:
# Επιλέγουμε τις στήλες που πρέπει να είναι αριθμητικές
numeric_cols = df.columns.difference(['Date', 'Time'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Interpolation μόνο στις αριθμητικές στήλες για να αποφύγουμε το FutureWarning
df[numeric_cols] = df[numeric_cols].interpolate(method='linear')
# Αν έχουν μείνει NaN στην αρχή, τα κλείνουμε με ffill/bfill
df.ffill(inplace=True)
df.bfill(inplace=True)

# Δημιουργία Datetime Index και διαγραφή των παλιών στηλών
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
df.set_index('datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

print("Ο καθαρισμός ολοκληρώθηκε. Missing values:", df.isna().sum().sum())


Ο καθαρισμός ολοκληρώθηκε. Missing values: 0


Feature Engineering (Sub_metering_4 & Resampling)

In [19]:

# Υπολογισμός Sub_metering_4 (σε Wh/min) και clip στο 0
df['Sub_metering_4'] = (df['Global_active_power'] * 1000 / 60) - df['Sub_metering_1'] - df['Sub_metering_2'] - df['Sub_metering_3']
df['Sub_metering_4'] = df['Sub_metering_4'].clip(lower=0)

# 1. Υπολογισμός Peak Hour Power (Η μέγιστη ωριαία κατανάλωση μέσα στη μέρα)
hourly_consumption = df['Global_active_power'].resample('h').sum()
peak_hour_series = hourly_consumption.resample('D').max()

# 2. Υπολογισμός Night Usage (22:00 - 06:00)
night_mask = (df.index.hour >= 22) | (df.index.hour < 6)
night_usage_series = df[night_mask]['Global_active_power'].resample('D').sum()

# Resampling σε ημερήσιο επίπεδο (Daily)
df_daily = df.resample('D').agg({
    'Global_active_power': 'sum',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum',
    'Sub_metering_4': 'sum'
})

# Μετατροπή συνολικής ισχύος σε Wh
df_daily['Daily_total_power'] = df_daily['Global_active_power'] * (1000 / 60)
df_daily['Peak_hour_power'] = peak_hour_series
df_daily['Night_usage_power'] = night_usage_series

Εμπλουτισμός (Lags, Time Features, One-Hot)

In [20]:
# Lag Features & Rolling Means
df_daily['Power_lag_1'] = df_daily['Daily_total_power'].shift(1)
df_daily['Power_lag_7'] = df_daily['Daily_total_power'].shift(7)
df_daily['Power_rolling_mean_3'] = df_daily['Daily_total_power'].shift(1).rolling(window=3).mean()
df_daily['Power_rolling_mean_7'] = df_daily['Daily_total_power'].shift(1).rolling(window=7).mean()
df_daily.dropna(inplace=True) # Αφαίρεση των πρώτων 7 ημερών λόγω shift

# Ημερολογιακά Χαρακτηριστικά
df_daily['Day_of_week'] = df_daily.index.dayofweek
df_daily['Is_workday'] = (df_daily['Day_of_week'] < 5).astype(int)
df_daily['Is_weekend'] = (df_daily['Day_of_week'] >= 5).astype(int)
df_daily['Month'] = df_daily.index.month

# One-Hot Encoding για Εποχές
def get_season(m):
    if m in [12,1,2]: return 'Winter'
    elif m in [3,4,5]: return 'Spring'
    elif m in [6,7,8]: return 'Summer'
    else: return 'Autumn'

df_daily['Season'] = df_daily['Month'].apply(get_season)
df_daily = pd.get_dummies(df_daily, columns=['Season'], prefix='Is')

# One-Hot Encoding για Ημέρες & Μήνες (Προαιρετικά αλλά καλό)
df_daily = pd.get_dummies(df_daily, columns=['Day_of_week'], prefix='Day')
df_daily = pd.get_dummies(df_daily, columns=['Month'], prefix='Month')

Έλεγχος Θορύβου (Outliers)

In [21]:
# Έλεγχος με Z-score
z_scores = stats.zscore(df_daily['Daily_total_power'])
outliers_count = len(df_daily[abs(z_scores) > 3])
print(f"Εντοπίστηκαν {outliers_count} ακραίες τιμές (Z-score > 3).")

Εντοπίστηκαν 12 ακραίες τιμές (Z-score > 3).


In [22]:
# Έλεγχος για μέρες με μηδενική κατανάλωση
zero_days = df_daily[df_daily['Daily_total_power'] == 0]
print(f"Μέρες με μηδενική κατανάλωση: {len(zero_days)}")

# Έλεγχος για ακραία Voltage
voltage_issues = df_daily[(df_daily['Voltage'] < 220) | (df_daily['Voltage'] > 260)]
print(f"Μέρες με ασυνήθιστη τάση: {len(voltage_issues)}")

Μέρες με μηδενική κατανάλωση: 0
Μέρες με ασυνήθιστη τάση: 0


Κανονικοποίηση (Scaling) & Target Variable

In [23]:
cols_to_scale = [
    'Daily_total_power', 'Global_reactive_power', 'Voltage',
    'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
    'Sub_metering_3', 'Sub_metering_4', 'Peak_hour_power',
    'Night_usage_power', 'Power_lag_1', 'Power_lag_7',
    'Power_rolling_mean_3', 'Power_rolling_mean_7'
]

scaler = StandardScaler()
df_daily_scaled = df_daily.copy()

# Εφαρμογή Scaling
df_daily_scaled[cols_to_scale] = scaler.fit_transform(df_daily[cols_to_scale])
print("Στατιστικά κανονικοποιημένων δεδομένων (πρέπει mean=0, std=1):")
display(df_daily_scaled[cols_to_scale].describe().loc[['mean', 'std']].round(2))

# Target Variable για Classification
mean_val = df_daily['Daily_total_power'].mean()
df_daily_scaled['High_Consumption'] = (df_daily['Daily_total_power'] > mean_val).astype(int)
print(f"\nΤο threshold ορίστηκε στα {mean_val:.2f} Wh.")

Στατιστικά κανονικοποιημένων δεδομένων (πρέπει mean=0, std=1):


,Daily_total_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4,Peak_hour_power,Night_usage_power,Power_lag_1,Power_lag_7,Power_rolling_mean_3,Power_rolling_mean_7
mean,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0



Το threshold ορίστηκε στα 26107.53 Wh.


In [24]:
# Αποθήκευση
df_daily_scaled.to_csv('household_power_final_preprocessed.csv', index=True)
print("Η προεπεξεργασία ολοκληρώθηκε! Το αρχείο είναι έτοιμο.")

Η προεπεξεργασία ολοκληρώθηκε! Το αρχείο είναι έτοιμο.
